# Cifrado Flujo

## Ejercicio 1

Escribe una función que determine si una secuencia de bits cumple los postulados de Golomb.

In [1]:
bits = [0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1]

def first_rule(seq):
    # Contamos el número de unos
    n_ones = sum(seq)
    n_bits = len(seq)
    # Si la longitud de la secuencia es 0
    if n_bits % 2 == 0:
        # a la fuerza tienen que ser iguales
        # num_ceros y num_unos
        return n_bits / 2 == n_ones
    else:
        n2_bits = n_bits // 2
        return n2_bits == n_ones or (n2_bits + 1) == n_ones

    
first_rule(bits)

True

En esta primera regla, podemos hacer una rápida comprobación si sumamos la secuencia de bits y obtenemos su longitud. Si la longitud es par, tenemos que el número de unos tiene que ser $\frac{|| bits ||}{2}$, y en caso de qeu sea impar, el número de unos tiene que ser igual a $\left\lfloor\frac{|| bits ||}{2}\right\rfloor$ o $\left\lceil\frac{|| bits ||}{2}\right\rceil$. En caso contrario, devolveremos un `False`.

In [2]:
from itertools import groupby
from collections import Counter

# Esta función calcula el número de rachas
# de longitud k en la secuencia seq      
def second_rule(seq):
    # obtenemos todas las rachas posibles que existen en la secuencia
    runs = [list(g) for k, g in groupby(seq)]
    # y contamos el número de rachas que hay para cada longitud
    count = Counter(map(lambda x: len(x), runs))
    last_key = -1
    # comprobamos si se cumple que #runs(k+1) == runs(k)
    for key, elem in count.items():
        if last_key != -1:      
            if count[last_key] < elem:  # en el caso de que el siguiente elemento
                return False            # no sea 1/2 veces más pequeño
            last_key = key
        else:
            last_key = key
            
    else:
        # si todo va bien, devolvemos True
        return True

second_rule(bits)

True

Para la segunda regla, comprobamos el número de rachas de longitud $k$ debe ser el doble que el número de rachas de longitud $k+1$ en la secuencia de bits, aunque esta condición se relaja un poco. En caso de que en algún momento esto no se cumpla, se devuelve `False`.

In [3]:
from numpy import bitwise_xor, nonzero

def rotate(seq, length):
    seq = [seq[-1]] + seq[:length-1]

def third_rule(seq):
    length, rotated_seq = len(seq), seq
    rotate(rotated_seq, length)
    # Calculamos la distancia hamming como un xor entre
    # ambas cadenas, y contamos los bits no nulos
    norm = len(bitwise_xor(seq, rotated_seq).nonzero()[0])
    for i in range(1, length):
        rotate(rotated_seq, length)
        if norm != len(bitwise_xor(seq, rotated_seq).nonzero()[0]):
            return False
    else:
        return True

third_rule(bits)

True

En esta regla, comprobamos si, rotando la cadena de bits, se mantiene constante la distancia Hamming. Esta distancia Hamming se calcula como la suma de los bits no nulos resultantes de hacer un $xor$ entre la cadena original y la cadena rotada.

## Ejercicio 2

Implementa registros lineales de desplazamiento con retroalimentación. La entrada son los coeficientes del polinomio de conexión, la semilla, y la longitud de la secuencia de salida.

Ilustra con ejemplos la dependencia del periodo de la semilla en el caso de polinomios reducibles, la independencia en el caso de polinomios irreducibles, y la maximalidad del periodo en el caso de polinomios primitivos.

Comprueba que los ejemplos con polinomios primitivos satisfacen los postulados de Golomb